In [5]:
import numpy as np
import pandas as pd

md = pd.read_csv('./kaggle/input/movies_metadata.csv', low_memory=False)

In [6]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [8]:
from ast import literal_eval

md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [9]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()

In [10]:
m = vote_counts.quantile(0.95)

In [11]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [12]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [13]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [15]:
qualified = qualified.sort_values('wr', ascending=False)

In [17]:
qualified.head(100)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
...,...,...,...,...,...,...,...
14825,Shutter Island,2010,6559,7,15.813629,"[Drama, Thriller, Mystery]",6.891075
31865,The Revenant,2015,6558,7,23.501228,"[Western, Drama, Adventure, Thriller]",6.891059
12704,WALL·E,2008,6439,7,16.088366,"[Animation, Family]",6.889173
13746,The Hangover,2009,6324,7,23.947351,[Comedy],6.887287


In [18]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2744528521.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [19]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [21]:
build_chart('Comedy').head()

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.463024
351,Forrest Gump,1994,8147,8,48.307194,7.963363
1225,Back to the Future,1985,6239,8,25.778509,7.952358
18465,The Intouchables,2011,5410,8,16.086919,7.945207
22841,The Grand Budapest Hotel,2014,4644,8,14.442048,7.936384


In [23]:
links_small = pd.read_csv('./kaggle/input/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [24]:
md = md.drop([19730, 29503, 35587])

In [25]:
md['id'] = md['id'].astype('int')

In [26]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [27]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/3912728496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['tagline'] = smd['tagline'].fillna('')
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/3912728496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['overview'] + smd['tagline']
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/3912728496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [31]:
tfidf_matrix.shape

(9099, 268124)

In [32]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [34]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [35]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [40]:
get_recommendations('The Matrix').head(10)

6364                    Pulse
2561                Supernova
7393                   Avatar
8497         The Zero Theorem
149                   Hackers
538     Hellraiser: Bloodline
5911            The Animatrix
3114             The Specials
4741                 Commando
1130                 Sneakers
Name: title, dtype: object

In [41]:
credits = pd.read_csv('./kaggle/input/credits.csv')
keywords = pd.read_csv('./kaggle/input/keywords.csv')

In [42]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [43]:
md.shape

(45463, 25)

In [44]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [45]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [46]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/3047342568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(literal_eval)
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/3047342568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['crew'] = smd['crew'].apply(literal_eval)
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/3047342568.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [47]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [48]:
smd['director'] = smd['crew'].apply(get_director)

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/4176871582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['crew'].apply(get_director)


In [49]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/864329085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/864329085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)


In [50]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/4209080348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [51]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/997285517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [52]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1272965743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1272965743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['director'].apply(lambda x: [x,x, x])


In [53]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2163307055.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


In [54]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [55]:
s = s[s > 1]

In [57]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [58]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [59]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1613546145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(filter_keywords)
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1613546145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1613546145.py:3: SettingWithCopyWarning: 
A value is t

In [60]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1585552675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/1585552675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))


In [62]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [63]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [65]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [68]:
get_recommendations('Alien').head(10)

8132                  Prometheus
3630                  Black Rain
1718                      Legend
485                 Blade Runner
8983                 The Martian
7119                Body of Lies
8512               The Counselor
8805      Exodus: Gods and Kings
3324                    Hannibal
2389    Someone to Watch Over Me
Name: title, dtype: object

In [69]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [70]:
improved_recommendations('The Dark Knight')

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2657751039.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2657751039.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2657751039.py:16: SettingWithCopyWarning: 
A value is try

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [71]:
improved_recommendations('The Matrix')

/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2657751039.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2657751039.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
/var/folders/cv/yn6d0z750g1bgwxl0vn4gmrjlgtmsm/T/ipykernel_76766/2657751039.py:16: SettingWithCopyWarning: 
A value is try

,title,vote_count,vote_average,year,wr
1011,The Terminator,4208,7,1984,6.835908
5544,"I, Robot",3889,6,2004,5.924193
4651,The Matrix Reloaded,3500,6,2003,5.916697
4928,The Matrix Revolutions,3155,6,2003,5.908689
7764,TRON: Legacy,2895,6,2010,5.901558
5636,Resident Evil: Apocalypse,1286,6,2004,5.809468
7424,Surrogates,1219,5,2009,5.064298
4739,Terminator 3: Rise of the Machines,2177,5,2003,5.040707
7296,Terminator Salvation,2496,5,2009,5.036275
8854,Terminator Genisys,3677,5,2015,5.025854


In [78]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


In [80]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

reader = Reader()

In [83]:
ratings = pd.read_csv('./kaggle/input/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [84]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, cross_validate

# Define the reader with the appropriate rating scale
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load dataset from your DataFrame
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train and test the model
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# Optionally, cross-validate
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8945  0.8989  0.8930  0.8975  0.8998  0.8967  0.0026  
MAE (testset)     0.6866  0.6908  0.6901  0.6930  0.6909  0.6903  0.0021  
Fit time          0.42    0.43    0.43    0.39    0.46    0.43    0.02    
Test time         0.05    0.04    0.04    0.04    0.04    0.04    0.00    


{'test_rmse': array([0.89450822, 0.8988601 , 0.89304141, 0.89748201, 0.89975207]),
 'test_mae': array([0.6865738 , 0.69081344, 0.69012406, 0.6930197 , 0.69089047]),
 'fit_time': (0.4201779365539551,
  0.4284353256225586,
  0.4273569583892822,
  0.3942699432373047,
  0.46406030654907227),
 'test_time': (0.04790806770324707,
  0.03731703758239746,
  0.035016775131225586,
  0.03563213348388672,
  0.03813290596008301)}

In [85]:
from surprise import SVD
from surprise.model_selection import cross_validate

# Define the SVD model
svd = SVD()

# Evaluate using cross-validation
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8987  0.9013  0.9024  0.8909  0.8974  0.8981  0.0040  
MAE (testset)     0.6915  0.6930  0.6923  0.6872  0.6913  0.6911  0.0020  
Fit time          0.42    0.43    0.43    0.44    0.47    0.44    0.02    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


In [86]:
from surprise import SVD

# Build the full training set
trainset = data.build_full_trainset()

# Define the SVD model
svd = SVD()

# Train the model
svd.fit(trainset)

In [87]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [88]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.727410581836572, details={'was_impossible': False})

In [89]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [90]:
id_map = pd.read_csv('./kaggle/input/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [91]:
indices_map = id_map.set_index('id')

In [92]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [94]:
hybrid(1, 'Aliens')

,title,vote_count,vote_average,year,id,est
1011,The Terminator,4208.0,7.4,1984,218,3.064771
987,Alien,4564.0,7.9,1979,348,3.050874
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.005148
922,The Abyss,822.0,7.1,1989,2756,2.981344
344,True Lies,1138.0,6.8,1994,36955,2.965611
7498,Daybreakers,646.0,6.0,2009,19901,2.907160
7939,Gantz,73.0,6.5,2011,56832,2.780923
1376,Titanic,7770.0,7.5,1997,597,2.729444
7828,I Am Number Four,1606.0,5.9,2011,46529,2.726972
6640,Déjà Vu,1519.0,6.6,2006,7551,2.707832
